### Import Dependencies

In [1]:
from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchAny, FusionQuery, Document


from langsmith import traceable, get_current_run_tree

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.types import Send, Command

from langchain_core.messages import AIMessage, ToolMessage, convert_to_openai_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional, Sequence
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import openai

import random
import ast
import inspect
import instructor
import json

from utils.utils import get_tool_descriptions, format_ai_message
from utils.tools import get_shopping_cart, add_to_shopping_cart, remove_from_cart, get_formatted_item_context, get_formatted_reviews_context

from langgraph.checkpoint.postgres import PostgresSaver

/Users/tanyakanodia/learning-ai/00-ai-engineering-bootcamp-cohort-2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create Coordinator Evaluation Dataset

In [2]:
class Delegation(BaseModel):
    agent: str
    task: str

class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: List[Delegation]
    final_answer: bool
    answer: str

In [3]:
def coordinator_agent(state):

   prompt_template =  """You are a Coordinator Agent as part of a shopping assistant.

Your role is to create plans for solving user queries and delegate the tasks accordingly.
You will be given a conversation history, your task is to create a plan for solving the user's query.
After the plan is created, you should output the next agent to invoke and the task to be performed by that agent.
Once an agent finishes its task, you will be handed the control back, you should then review the conversation history and revise the plan.
If there is a sequence of tasks to be performed by a single agent, you should combine them into a single task.

The possible agents are:

- product_qa_agent: The user is asking a question about a product. This can be a question about available products, their specifications, user reviews etc.
- shopping_cart_agent: The user is asking to add or remove items from the shopping cart or questions about the current shopping cart.

CRITICAL RULES:
- If next_agent is "", final_answer MUST be false
(You cannot delegate the task to an agent and return to the user in the same response)
- If final_answer is true, next_agent MUST be ""
(You must wait for agent results before returning to user)
- If you need to call other agents before answering, set:
next_agent="...", final_answer=false
- After receiving agent results, you can then set:
next_agent="", final_answer=true
- One of the following has to be true:
next_agent is "" and final_answer is true
next_agent is not "" and final_answer is false

Additional instructions:

- Do not route to any agent if the user's query needs clarification. Do it yourself.
- Write the plan to the plan field.
- Write the next agent to invoke to the next_agent field.
- Once you have all the information needed to answer the user's query, you should set the final_answer field to True and output the answer to the user's query.
- The final answer to the user query should be a comprehensive answer that explains the actions that were performed to answer the query.
- Never set final_answer to true if the plan is not complete.
- You should output the next_agent field as well as the plan field.
"""

   template = Template(prompt_template)
   
   prompt = template.render()

   messages = state.messages

   conversation = []

   for message in messages:
        conversation.append(convert_to_openai_messages(message))

   client = instructor.from_openai(OpenAI())

   response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        response_model=CoordinatorAgentResponse,
        messages=[{"role": "system", "content": prompt}, *conversation],
        temperature=0,
   )

   if response.final_answer:
      ai_message = [AIMessage(
         content=response.answer,
      )]
   else:
      ai_message = []

   return {
      "messages": ai_message,
      "answer": response.answer,
      "coordinator_agent": {
         "iteration": state.coordinator_agent.iteration + 1,
         "final_answer": response.final_answer,
         "next_agent": response.next_agent,
         "plan": [data.model_dump() for data in response.plan]
      }
   }

In [4]:
class ToolCall(BaseModel):
    name: str
    arguments: dict

class RAGUsedContext(BaseModel):
    id: str = Field(description="ID of the item used to answer the question.")
    description: str = Field(description="Short description of the item used to answer the question.")

class AgentProperties(BaseModel):
    iteration: int = 0
    final_answer: bool = False
    available_tools: List[Dict[str, Any]] = []
    tool_calls: List[ToolCall] = []

class CoordinatorAgentProperties(BaseModel):
    iteration: int = 0
    final_answer: bool = False
    plan: List[Delegation] = []
    next_agent: str = ""

class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    user_intent: str = ""
    product_qa_agent: AgentProperties = Field(default_factory=AgentProperties)
    shopping_cart_agent: AgentProperties = Field(default_factory=AgentProperties)
    coordinator_agent: CoordinatorAgentProperties = Field(default_factory=AgentProperties)
    answer: str = ""
    references: Annotated[List[RAGUsedContext], add] = []
    user_id: str = ""
    cart_id: str = ""

In [5]:
initial_state = State(
    messages=[{"role": "user", "content": "What is the weather today?"}]
)

In [6]:
answer = coordinator_agent(initial_state)

In [7]:
answer

{'messages': [],
 'answer': 'I cannot provide weather information. Please ask about shopping-related queries or product information.',
 'coordinator_agent': {'iteration': 1,
  'final_answer': False,
  'next_agent': '',
  'plan': []}}

In [8]:
initial_state = State(
    messages=[{"role": "user", "content": "Can I get some earphones?"}]
)

In [9]:
answer = coordinator_agent(initial_state)

In [10]:
answer

{'messages': [],
 'answer': '',
 'coordinator_agent': {'iteration': 1,
  'final_answer': False,
  'next_agent': 'product_qa_agent',
  'plan': [{'agent': 'product_qa_agent',
    'task': 'Find available earphones products'}]}}

In [11]:
coordinator_eval_dataset = [
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "What is the weather today?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can I get some earphones?"}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add an item with ID B09NLTDHQ6 to my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "shopping_cart_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add those earphones to my cart?"}
            ]
        }, 
        # this is set to final answer - true because in first delegation the agent has no context of which earphones. 
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add the best items to my cart? I am looking for laptop bags."}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you find some good reviews for items in my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "shopping_cart_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you put the items with the most positive user reviews to my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "What kind of stuff do you sell?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you help me with my order?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add two, ideally red tablets to my cart?"}
            ]
        },
        "outputs": {
        # this is set to final answer - true because in first delegation the agent just answers the query and no other agent is invoked. 
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    }
]

### Upload the dataset to LangSmith

In [13]:
from langsmith import Client
import os

client = Client(api_key=os.environ["LANGCHAIN_API_KEY"])

dataset_name = "coordinator-eval-dataset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Dataset for evaluating routing of the coordinator agent"
)

In [14]:
for item in coordinator_eval_dataset:
    client.create_example(
        dataset_id=dataset.id,
        inputs={"messages": item["inputs"]["messages"]},
        outputs={
            "next_agent": item["outputs"]["next_agent"],
            "coordinator_final_answer": item["outputs"]["coordinator_final_answer"]
        }
    )

In [15]:
def next_agent_evaluator(run, example):

    next_agent_match = run.outputs["coordinator_agent"]["next_agent"] == example.outputs["next_agent"] 
    final_answer_match = run.outputs["coordinator_agent"]["final_answer"] == example.outputs["coordinator_final_answer"]
    
    return next_agent_match and final_answer_match

In [16]:
results = client.evaluate(
    lambda x: coordinator_agent(State(messages=x["messages"])),
    data="coordinator-eval-dataset",
    evaluators=[
        next_agent_evaluator
    ],
    experiment_prefix="coordinator-eval-dataset"
)

View the evaluation results for experiment: 'coordinator-eval-dataset-505662c0' at:
https://smith.langchain.com/o/eda70326-53b4-40a9-8f01-1ae7702e6112/datasets/457e4b89-ef35-4823-a10c-09acc5e64afb/compare?selectedSessions=d9010e20-d489-48eb-aa43-d1461b3b7207




10it [00:12,  1.26s/it]
